In [1]:
import numpy as np

import rotations
import test_rotations

## Testing conversions between different representations

### Roundtrips between unit quaternions and rotation matrices

In [2]:
for i in range(1000):
    test_rotations.test_matrix_quaternion()

In [3]:
for i in range(1000):
    test_rotations.test_quaternion_matrix()

AssertionError: 

### Roundtrips between unit quaternions and Euler vectors

In [4]:
for i in range(1000):
    test_rotations.test_quaternion_euler()

In [10]:
for i in range(1000):
    test_rotations.test_euler_quaternion()
    
# Note: Only works when Euler vector has -2pi < theta < +2pi

### Full roundtrips between rotation matrices and Euler vectors

In [9]:
for i in range(1000):
    test_rotations.test_euler_matrix()

known_euler [-0.50573812 -1.72650898  0.6661441 ]
euler [ 1.15064526  3.92811871 -1.51559774]


AssertionError: 

In [6]:
for i in range(1000):
    test_rotations.test_matrix_euler()

### Other tests

In [7]:
test_rotations.test_quaternion_from_euler()

In [8]:
for i in range(1000):
    test_rotations.test_matrix_from_euler()

## Rotations greater than pi are broken!!!

In [15]:
def matrix_from_euler(euler):
    quaternion = rotations.quaternion_from_euler(euler)
    if np.linalg.norm(euler) > np.pi:
        assert quaternion[0] < 0
        print("yeah")
        #quaternion *= -1
    matrix = rotations.matrix_from_quaternion(quaternion)
    return matrix

In [12]:
def euler_from_matrix(matrix):
    quaternion = rotations.quaternion_from_matrix(matrix)
    if quaternion[0] < 0:
        print(np.arcsin(np.linalg.norm(quaternion[1:])))
    euler = rotations.euler_from_quaternion(quaternion)
    return euler

In [16]:
def test_euler_matrix():
    while True:
        known_euler = np.random.normal(loc=0.0, scale=np.pi, size=3)
        if np.linalg.norm(known_euler) < 2*np.pi:
            break

    euler = euler_from_matrix(matrix_from_euler(known_euler))

    check = np.allclose(known_euler, euler)

    if not check:
        print("known_euler", known_euler)
        print('norm of known_euler / 2pi', np.linalg.norm(known_euler)/(2*np.pi))
        print('...', np.cos(np.linalg.norm(known_euler))/2)
        print("euler", euler)

    assert check

#     assert np.allclose(rotations.euler_from_quaternion( \
#                        rotations.quaternion_from_matrix( \
#                        rotations.matrix_from_quaternion( \
#                        rotations.quaternion_from_euler(np.zeros((3), dtype=float))))),
#                        np.zeros((3), dtype=float))

In [75]:
test_euler_matrix()

yeah
known_euler [-2.98746165  2.52771214  1.09623465]
norm of known_euler / 2pi 0.646803537838
... -0.301956943025
euler [ 1.63134681 -1.38029391 -0.59861484]


AssertionError: 

## Interpolation of rotations